## 19/02/2019

Reformatting the data

In [1]:
import numpy as np
import pandas as pd
import pdb

X = pd.read_pickle('/Users/matthewashman/github/MasterProject2018/Data/X_all.pkl')

In [2]:
X[X['Type']=='avnrt'].head()

,Channel,Coupling Interval,Data,Patient,S1/S2,Type,Label
1956,CS1-2,360,"[94, 192, 272, 268, 425, 550, 697, 911, 639, 2...",1,S2,avnrt,NaN
1957,CS1-2,360,"[229.0, 305.0, 297.0, 392.0, 531.0, 499.0, 806...",1,S1,avnrt,NaN
1958,CS1-2,360,"[205.0, 130.0, 221.0, 376.0, 439.0, 386.0, 732...",1,S1,avnrt,NaN
1959,CS3-4,360,"[105, 429, 653, 987, 1723, 2797, 3607, -2015, ...",1,S2,avnrt,NaN
1960,CS3-4,360,"[78.0, 156.0, 578.0, 1385.0, 2468.0, 3856.0, 5...",1,S1,avnrt,NaN


In [3]:
af_patients = X[X['Type']=='af']['Patient'].unique()
at_patients = X[X['Type']=='at']['Patient'].unique()
avnrt_patients = X[X['Type']=='avnrt']['Patient'].unique()
avrt_patients = X[X['Type']=='avrt']['Patient'].unique()
ep_patients = X[X['Type']=='ep']['Patient'].unique()

patient_types = ('af', 'at', 'avnrt', 'avrt', 'ep')
all_patients = (af_patients, at_patients, avnrt_patients, avrt_patients, ep_patients)

In [5]:
X_list = []
for patient_type, patients in zip(patient_types, all_patients):
    for patient in patients:
        for pulse_type in ('S1', 'S2'):
            patient_X = X[(X['Type']==patient_type) & (X['Patient']==patient) & (X['S1/S2']==pulse_type)]
            cis = patient_X['Coupling Interval'].unique()
            for ci in cis:
                ci_patient_X = patient_X[patient_X['Coupling Interval']==ci]
                # Extract data
                if 1 in np.float64(ci_patient_X['Label'].values):
                    label = 1
                elif 0 in np.float64(ci_patient_X['Label'].values):
                    label = 0
                else:
                    label = np.nan

                # Sometimes 2 rows of data if S1.
                cs12_datas = np.squeeze(ci_patient_X[ci_patient_X['Channel']=='CS1-2']['Data'])
                cs34_datas = np.squeeze(ci_patient_X[ci_patient_X['Channel']=='CS3-4']['Data'])
                cs56_datas = np.squeeze(ci_patient_X[ci_patient_X['Channel']=='CS5-6']['Data'])
                cs78_datas = np.squeeze(ci_patient_X[ci_patient_X['Channel']=='CS7-8']['Data'])

                if isinstance(cs12_datas, pd.core.series.Series):
                    for cs12_data, cs34_data, cs56_data, cs78_data in zip(cs12_datas, cs34_datas, cs56_datas, cs78_datas):
                        temp_dict = {}
                        temp_dict.update({'Type':patient_type, 'Patient':patient, 'Coupling Interval':ci, 'S1/S2':pulse_type,
                                          'Label':label, 'CS1-2':cs12_data, 'CS3-4':cs34_data, 'CS5-6':cs56_data, 
                                          'CS7-8':cs78_data});
                        X_list.append(temp_dict)
                else:
                    temp_dict = {}
                    temp_dict.update({'Type':patient_type, 'Patient':patient, 'Coupling Interval':ci, 'S1/S2':pulse_type,
                                      'Label':label, 'CS1-2':cs12_datas, 'CS3-4':cs34_datas, 'CS5-6':cs56_datas, 
                                      'CS7-8':cs78_datas})
                    X_list.append(temp_dict)
                
                
X_reformatted = pd.DataFrame(X_list)

In [11]:
X_reformatted.to_pickle('/Users/matthewashman/github/MasterProject2018/Data/X_all_compact.pkl')